작업형2-1. 데이터 분석 연습하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
#0. 문제의도 파악
#고객 데이터 활용하여 '성별 예측값(분류모델, predict_proba)' 구하기, roc_auc_score

#1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

#2. 데이터 로드
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/x_train.csv', encoding= 'CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/x_test.csv', encoding= 'CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/y_train.csv', encoding= 'CP949')

#3. 전처리
#cust_id: 삭제 후 저장해두기, 환불금액: 파생변수, 주구매상품&주구매지점: 라벨 인코딩, 표준화 크기변환 필요
#불필요한 칼럼 삭제: cust_id
x_test_cust_id = x_test['cust_id']

x_train = x_train.drop(columns= 'cust_id')
x_test = x_test.drop(columns= 'cust_id')
y_train = y_train.drop(columns= 'cust_id')

#라벨 인코딩: 주구매상품, 주구매지점
encoder = LabelEncoder()
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])

x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])

#파생변수 만들기: 환불금액 (test에 대한 condition 따로 만들면 x: test는 '처음 보는 것'이라는 가정!!)
condition = x_train['환불금액'] > 0
x_train.loc[condition, '환불금액_new'] = 1
x_train.loc[~condition, '환불금액_new'] = 0
x_train = x_train.drop(columns= '환불금액')

x_test.loc[condition, '환불금액_new'] = 1
x_test.loc[~condition, '환불금액_new'] = 0
x_test = x_test.drop(columns= '환불금액')

#표준화 크기변환
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns= x_test.columns)

#상관관계 확인
data = pd.concat([x_train, y_train], axis=1)
x_train = x_train.drop(columns= '최대구매액')
x_test = x_test.drop(columns= '최대구매액')

#4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#5. 모델 생성 및 하이퍼 파라미터
model_dt = DecisionTreeClassifier(criterion= 'entropy', max_depth=5, random_state=42)
model_lr = LogisticRegression(random_state=42, max_iter=100)
model_xgb = XGBClassifier(n_estimators=200, max_depth=5, eval_metric='error', random_state=42)

#6. 모델 학습 및 평가
model_dt.fit(X_TRAIN, Y_TRAIN)
model_lr.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

PRED_DT = model_dt.predict_proba(X_TEST)
PRED_LR = model_lr.predict_proba(X_TEST)
PRED_XGB = model_xgb.predict_proba(X_TEST)
Y_TEST_PRED = ((PRED_DT + PRED_LR + PRED_XGB)/3)[:,1]
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

#7. 결과 예측
pred_dt = model_dt.predict_proba(x_test)
pred_lr = model_lr.predict_proba(x_test)
pred_xgb = model_xgb.predict_proba(x_test)
y_test_pred = pd.DataFrame(((pred_dt + pred_lr + pred_xgb)/3)).rename(columns= {1: 'gender'})['gender']

#8. 파일 저장
final = pd.concat([x_test_cust_id, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009_작업2_1.csv', index=False)
final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009_작업2_1.csv')
print(final)

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6671642175155055
      cust_id    gender
0        3500  0.350724
1        3501  0.144547
2        3502  0.400539
3        3503  0.419569
4        3504  0.388993
...       ...       ...
2477     5977  0.565121
2478     5978  0.539276
2479     5979  0.320556
2480     5980  0.453798
2481     5981  0.522191

[2482 rows x 2 columns]


작업형2-2. 분류 모델 수행하기

In [68]:
##0. 문제의도 파악
#고객 정보로 '생존여부(분류모델)' 예측, roc_auc_score

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

##2. 파일 읽기
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_x_train.csv', encoding='CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_x_test.csv', encoding= 'CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/titanic_y_train.csv', encoding= 'CP949')

##3. 전처리
#PassengerId: 따로 저장해두기, 승객이름&객실번호: 삭제, 성별: 수동 인코딩,
#형제자매배우자수&부모자식수: 파생변수, 선착장: 원핫 인코딩, 나이&티켓번호: 상관관계 확인 후 처리(제거)

#불필요한 칼럼 삭제: passengerid, 승객이름, 객실번호
x_test_passengerid = x_test['PassengerId']

x_train = x_train.drop(columns= ['PassengerId', '승객이름', '객실번호'])
x_test = x_test.drop(columns= ['PassengerId', '승객이름', '객실번호'])
y_train = y_train.drop(columns= ['癤풮assengerId'])

#수동 인코딩: 성별
x_train = x_train.replace('male', 0).replace('female', 1)
x_test = x_test.replace('male', 0).replace('female', 1)

#파생변수: 형제자매배우자수 & 부모자식수
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_train = x_train.drop(columns= ['형제자매배우자수', '부모자식수'])
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_test = x_test.drop(columns= ['형제자매배우자수', '부모자식수'])

#원핫 인코딩: 선착장
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first=True).rename(columns= {'Q': '선착장Q', 'S': '선착장S'})
x_train = pd.concat([x_train, 선착장_dummy], axis=1)
x_train = x_train.drop(columns= '선착장')

선착장_dummy = pd.get_dummies(x_test['선착장'], drop_first=True).rename(columns= {'Q':'선착장Q', 'S':'선착장S'})
x_test = pd.concat([x_test, 선착장_dummy], axis=1)
x_test = x_test.drop(columns= '선착장')

#상관관계 확인: 나이, 티켓번호
data = pd.concat([x_train, y_train], axis=1)

x_train = x_train.drop(columns= ['나이', '티켓번호'])
x_test = x_test.drop(columns= ['나이', '티켓번호'])

#테스트 데이터 확인
x_test['운임요금'] = x_test['운임요금'].fillna(0)

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_rf = RandomForestClassifier(n_estimators=100, criterion= 'entropy', max_depth=10, random_state=42)
model_lr = LogisticRegression(random_state=42)
model_xgb = XGBClassifier(max_depth=5, n_estimators=200, eval_metric= 'error', random_state=42)

##6. 모델 학습 및 평가
model_rf.fit(X_TRAIN, Y_TRAIN)
model_lr.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

best_models = [('rf', model_rf), ('lr', model_lr), ('xgb', model_xgb)]
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = voting_clf.predict(X_TEST)
# print(roc_auc_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측
y_test_pred = pd.DataFrame(voting_clf.predict(x_test)).rename(columns= {0:'Survived'})
final = pd.concat([x_test_passengerid, y_test_pred], axis=1)

##8. 파일 제출
final.to_csv('/content/drive/MyDrive/bigData-main/작업2_2.csv', index=False)

작업형2-3. 예측 모델 수행하기

In [99]:
##0. 문제의도 파악
#고객 속성 통해 '자전거 대여량(연속변수; 예측모델)' 예측모델 만들기, r2_score

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

##2. 파일 읽기
x_train = pd.read_csv('/content/drive/MyDrive/bigData-main/bike_x_train.csv', encoding= 'CP949')
x_test = pd.read_csv('/content/drive/MyDrive/bigData-main/bike_x_test.csv', encoding= 'CP949')
y_train = pd.read_csv('/content/drive/MyDrive/bigData-main/bike_y_train.csv', encoding='CP949')
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)

# print(x_train.info())
# print(x_train.describe())

##3. 전처리
#datetime: 연/월/일/요일/시간으로 분해, 계절&공휴일&근무일&날씨: 범주형 변수 값 확인&상관관계 파악
#파생변수; 날짜형 변환, 새로운 칼럼 생성
x_train['datetime'] = pd.to_datetime(x_train['datetime'])

x_train['year'] = x_train['datetime'].dt.year
x_train['month'] = x_train['datetime'].dt.month
x_train['day'] = x_train['datetime'].dt.day
x_train['hour'] = x_train['datetime'].dt.hour
x_train['dayofweek'] = x_train['datetime'].dt.dayofweek
# print(x_train['year'].unique())
# print(x_train['month'].unique())
# print(x_train['day'].unique())
# print(x_train['hour'].unique())
# print(x_train['dayofweek'].unique())

x_test['datetime'] = pd.to_datetime(x_test['datetime'])

x_test['year'] = x_test['datetime'].dt.year
x_test['month'] = x_test['datetime'].dt.month
x_test['day'] = x_test['datetime'].dt.day
x_test['hour'] = x_test['datetime'].dt.hour
x_test['dayofweek'] = x_test['datetime'].dt.dayofweek

#불필요한 칼럼 삭제
x_test_datetime = x_test['datetime']
x_train = x_train.drop(columns= 'datetime')
x_test = x_test.drop(columns= 'datetime')
y_train = y_train.drop(columns= '癤풼atetime')
# print(x_train.head())

#범주형 변수 값 확인: 계절, 공휴일, 근무일, 날씨
# print(x_train['계절'].unique())
# print(x_train['공휴일'].unique())
# print(x_train['근무일'].unique())
# print(x_train['날씨'].unique())
data = pd.concat([x_train, y_train], axis=1)

#독립변수와 종속변수 간 관계 확인: 차이 없는 것 삭제(day, dayofweek)
# print(data.groupby(['계절'])['count'].sum())
# print(data.groupby(['공휴일'])['count'].sum())
# print(data.groupby(['근무일'])['count'].sum())
# print(data.groupby(['날씨'])['count'].sum())

# print(data.groupby(['year'])['count'].sum())
# print(data.groupby(['month'])['count'].sum())
# print(data.groupby(['day'])['count'].sum())
# print(data.groupby(['hour'])['count'].sum())
# print(data.groupby(['dayofweek'])['count'].sum())

x_train = x_train.drop(columns= ['day', 'dayofweek'])
x_test = x_test.drop(columns= ['day', 'dayofweek'])

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
# print(X_TRAIN.shape)
# print(X_TEST.shape)
# print(Y_TRAIN.shape)
# print(Y_TEST.shape)

##5. 모델 생성 및 하이퍼 파라미터
model_lr = LinearRegression()
model_rf = RandomForestRegressor(random_state=42)
model_gbr = GradientBoostingRegressor(random_state=42)
model_xgb = XGBRegressor(max_depth=10, random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_lr.fit(X_TRAIN, Y_TRAIN)
model_rf.fit(X_TRAIN, Y_TRAIN)
model_gbr.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

# pred_lr = model_lr.predict(X_TEST)
# pred_rf = model_rf.predict(X_TEST)
# pred_gbr = model_gbr.predict(X_TEST)
# pred_xgb = model_xgb.predict(X_TEST)
# print(r2_score(Y_TEST, pred_lr))
# print(r2_score(Y_TEST, pred_rf))
# print(r2_score(Y_TEST, pred_gbr))
# print(r2_score(Y_TEST, pred_xgb))

best_models = [('rf', model_rf), ('gbr', model_gbr), ('xgb', model_xgb)]
voting_rg = VotingRegressor(estimators=best_models)
voting_rg.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = pd.DataFrame(voting_rg.predict(X_TEST)).rename(columns= {0: 'count'})
Y_TEST_PRED[Y_TEST_PRED['count'] < 0] = 0
# print(r2_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측
y_test_pred = pd.DataFrame(voting_rg.predict(x_test)).rename(columns= {0: 'count'})
y_test_pred[y_test_pred['count'] < 0] = 0

final = pd.concat([x_test_datetime, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009작업2_3.csv', index=False)

##8. 파일 제출
# final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009작업2_3.csv')
# print(final)

[09:00:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[09:00:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Datamanim 모의고사 1회차

작업형2

In [37]:
##0. 문제 의도 파악
#은행 텔레마케팅에 대한 고객 반응여부(분류모델), roc_auc_score

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

##2. 파일 읽기
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

x_train = train.drop(columns= 'y')
y_train = train['y']

##3. 전처리
#ID: 따로 저장, 범주형(job, marital, education, default, housing, loan, contact, month, poutcome)

#ID 따로 저장
x_test_id = x_test['ID']

x_train = x_train.drop(columns= 'ID')
x_test = x_test.drop(columns= 'ID')
y_train = y_train.drop(columns= 'ID')

#범주형 변수 값 확인: unique()

#job & education & poutcome & month: 라벨, marital & contact: 원핫, default & housing & loan: 수동인코딩
#수동 인코딩
x_train['default'] = x_train['default'].replace('no', 0).replace('yes', 1)
x_test['default'] = x_test['default'].replace('no', 0).replace('yes', 1)

x_train['housing'] = x_train['housing'].replace('no', 0).replace('yes', 1)
x_test['housing'] = x_test['housing'].replace('no', 0).replace('yes', 1)

x_train['loan'] = x_train['loan'].replace('no', 0).replace('yes', 1)
x_test['loan'] = x_test['loan'].replace('no', 0).replace('yes', 1)

y_train = y_train.replace('no', 0).replace('yes', 1)

#라벨 인코딩
encoder = LabelEncoder()
x_train['job'] = encoder.fit_transform(x_train['job'])
x_test['job'] = encoder.fit_transform(x_test['job'])

x_train['education'] = encoder.fit_transform(x_train['education'])
x_test['education'] = encoder.fit_transform(x_test['education'])

x_train['poutcome'] = encoder.fit_transform(x_train['poutcome'])
x_test['poutcome'] = encoder.fit_transform(x_test['poutcome'])

x_train['month'] = encoder.fit_transform(x_train['month'])
x_test['month'] = encoder.fit_transform(x_test['month'])

#원핫 인코딩
marital_dummy = pd.get_dummies(x_train['marital'], drop_first=True).rename(columns= {'married':'marital_married', 'single':'marital_single'})
x_train = pd.concat([x_train, marital_dummy], axis=1)
marital_dummy = pd.get_dummies(x_test['marital'], drop_first=True).rename(columns= {'married':'marital_married', 'single':'marital_single'})
x_test = pd.concat([x_test, marital_dummy], axis=1)

contact_dummy = pd.get_dummies(x_train['contact'], drop_first=True).rename(columns= {'telephone':'contact_telephone', 'unknown':'contact_unknown'})
x_train = pd.concat([x_train, contact_dummy], axis=1)
contact_dummy = pd.get_dummies(x_test['contact'], drop_first=True).rename(columns= {'telephone':'contact_telephone', 'unknown':'contact_unknown'})
x_test = pd.concat([x_test, contact_dummy], axis=1)

#불필요한 칼럼 삭제
x_train = x_train.drop(columns= ['marital', 'contact'])
x_test = x_test.drop(columns= ['marital', 'contact'])

##4. 데이터 분리
y_train = pd.DataFrame(y_train).rename(columns= {'y':'predict'})
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_rf = RandomForestClassifier(max_depth=5, random_state=42)
model_xgb = XGBClassifier(max_depth=10, n_estimators=200, eval_metric='error', random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_rf.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

pred_rf = model_rf.predict(X_TEST)
pred_xgb = model_xgb.predict(X_TEST)

best_models = [('rf', model_rf), ('xgb', model_xgb)]
voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = pd.DataFrame(voting_clf.predict_proba(X_TEST))[1]
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측
y_test_pred = pd.DataFrame(voting_clf.predict_proba(x_test)).rename(columns= {0: 'predict'})

##8. 파일 제출
submission['predict'] = y_test_pred[1]
submission.to_csv('/content/drive/MyDrive/bigData-main/004001009모의고사1_작업2.csv', index=False)

0.7808505464272623
         ID   predict
0     53608  0.828560
1     51055  0.599995
2     52573  0.092527
3     50458  0.176170
4     52272  0.208626
...     ...       ...
4286  50332  0.171974
4287  51293  0.812921
4288  54023  0.226529
4289  53775  0.666631
4290  51945  0.154705

[4291 rows x 2 columns]


Datamanim 분류 연습문제
1. 서비스 이탈여부

In [77]:
##0. 문제의도 파악
#고객 신상정보 통해 회사 서비스 이탈 예측(분류모델)

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

##2. 파일 읽기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv')
# print(x_train.head())
# print(x_train.shape)
# print(x_test.head())
# print(x_test.shape)
# print(y_train.head())
# print(y_train.shape)
# print(y_test.head())
# print(y_test.shape)

##3. 전처리
# print(x_train.info())
# print(x_train.describe())
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())

#CustomerId: 따로 저장, Surname: 값 확인 후 삭제, Geography: 원핫 or라벨, Gender: 수동인코딩, 데이터 스케일링 필요
#불필요한 칼럼 삭제
x_test_customerid = x_test['CustomerId']
x_train = x_train.drop(columns= 'CustomerId')
x_test = x_test.drop(columns= 'CustomerId')
y_train = y_train.drop(columns= 'CustomerId')

# print(x_train.head())
# print(x_train.info())

#원핫: Geography /라벨 인코딩: Surname
# print(x_train['Surname'].value_counts())
# print(x_train['Geography'].value_counts())
# print(x_train['Gender'].unique())
Geography_dummy = pd.get_dummies(x_train['Geography'], drop_first=True).rename(columns= {'Germany': 'Geo_Germany', 'Spain':'Geo_Spain'})
x_train = pd.concat([x_train, Geography_dummy], axis=1)
x_train = x_train.drop(columns= 'Geography')
Geography_dummy = pd.get_dummies(x_test['Geography'], drop_first=True).rename(columns= {'Germany': 'Geo_Germany', 'Spain':'Geo_Spain'})
x_test = pd.concat([x_test, Geography_dummy], axis=1)
x_test = x_test.drop(columns= 'Geography')

encoder = LabelEncoder()
x_train['Surname'] = encoder.fit_transform(x_train['Surname'])
x_test['Surname'] = encoder.fit_transform(x_test['Surname'])

#수동인코딩: Gender
x_train['Gender'] = x_train['Gender'].replace('male',0).replace('Male',0).replace('female',1).replace('Female',1).replace(' male',0)
x_test['Gender'] = x_test['Gender'].replace('male',0).replace('Male',0).replace('female',1).replace('Female',1).replace(' male',0)

#파생변수//상관관계 확인
data = pd.concat([x_train, y_train], axis=1)
# print(data.corr())
#Tenure, EstimatedSalary 빼고 해보기
x_train = x_train.drop(columns= ['Tenure', 'EstimatedSalary'])
x_test = x_test.drop(columns= ['Tenure', 'EstimatedSalary'])

#표준화 크기변환
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test = pd.DataFrame(scaler.transform(x_test), columns= x_test.columns)

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
# print(X_TRAIN.shape)
# print(X_TEST.shape)
# print(Y_TRAIN.shape)
# print(Y_TEST.shape)

##5. 모델 생성 및 하이퍼 파라미터
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_xgb = XGBClassifier(n_estimators=100, max_depth=10, eval_metric='error',random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_rf.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

pred_rf = pd.DataFrame(model_rf.predict_proba(X_TEST)).iloc[:,1]
pred_xgb = pd.DataFrame(model_xgb.predict_proba(X_TEST)).iloc[:,1]

print(roc_auc_score(Y_TEST, pred_rf))
print(roc_auc_score(Y_TEST, pred_xgb))

best_models = [('rf', model_rf), ('xgb', model_xgb)]
voting_clf = VotingClassifier(estimators=best_models, voting='soft')
voting_clf.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = pd.DataFrame(voting_clf.predict_proba(X_TEST)).rename(columns= {0:'Exited'}).iloc[:,1]
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측
y_test_pred = pd.DataFrame(voting_clf.predict_proba(x_test)).rename(columns= {1:'Exited'}).iloc[:,1]
print(roc_auc_score(y_test['Exited'], y_test_pred))

##8. 파일 제출
final = pd.concat([x_test_customerid, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009분류1.csv', index=False)

final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009분류1.csv')
print(final)

0.8590258908333216
0.8379359135001198
0.8541088851032678
0.852969347695292
      CustomerId    Exited
0       15601012  0.874317
1       15734762  0.967556
2       15586757  0.045742
3       15590888  0.069680
4       15726087  0.396996
...          ...       ...
3496    15733966  0.526967
3497    15669994  0.100787
3498    15712403  0.914195
3499    15643819  0.021109
3500    15644962  0.019636

[3501 rows x 2 columns]


Datamanim 회귀 연습문제
1. 학생성적

In [109]:
##0. 문제의도 파악
#학생정보 통해 '학생성적' 예측하는 회귀모델

##1. 패키지 불러오기
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

##2. 파일 읽기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv')
y_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv')

##3. 전처리
#살펴볼 값: school, sex, address, famsize, Pstatus, Mjob, Fjob, reason, guardian
#schoolsup, famsup, paid, activities, nursery, higher, internet, romantic
#삭제(StudentID)
x_test_studentid = x_test['StudentID']
x_train = x_train.drop(columns= 'StudentID')
x_test = x_test.drop(columns= 'StudentID')
y_train = y_train.drop(columns= 'StudentID')
y_test = y_test.drop(columns= 'StudentID')

#범주형 변수 값 체크: unique()

#원핫
x_dummy = pd.get_dummies(pd.concat([x_train, x_test], axis=0))
len_split = x_train.shape[0]
x_train = x_dummy[:len_split]
x_test = x_dummy[len_split:]

##4. 데이터 분리
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

##5. 모델 생성 및 하이퍼 파라미터
model_rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
model_xgb = XGBRegressor(n_estimators=100, max_depth=10, random_state=42)

##6. 모델 학습 및 평가, 앙상블
model_rf.fit(X_TRAIN, Y_TRAIN)
model_xgb.fit(X_TRAIN, Y_TRAIN)

pred_rf = model_rf.predict(X_TEST)
pred_xgb = model_xgb.predict(X_TEST)
# print(r2_score(Y_TEST, pred_rf))
# print(r2_score(Y_TEST, pred_xgb))

Y_TEST_PRED = pd.DataFrame(model_rf.predict(X_TEST)).rename(columns= {0:'G3'})
# print(r2_score(Y_TEST, Y_TEST_PRED))

##7. 결과 예측
y_test_pred = pd.DataFrame(model_rf.predict(x_test)).rename(columns= {0:'G3'})
print(r2_score(y_test, y_test_pred))

# ##8. 파일 제출
final = pd.concat([x_test_studentid, y_test_pred], axis=1)
final.to_csv('/content/drive/MyDrive/bigData-main/004001009회귀1.csv', index=False)
# final = pd.read_csv('/content/drive/MyDrive/bigData-main/004001009회귀1.csv')
# print(final)

[17:14:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.821247018530205
